# TPM - Atividade 1

Com base na Planilha MOD 4 – Risco e Retorno em Renda Variável – Base de dados, recrie as carteiras da aba Portifólios, assumindo alocação de valores financeiros iguais no dia 30/12/2019.

In [1]:
import pandas as pd
import numpy as np
import re
import csv

from tqdm.notebook import tqdm

import config as cfg
import helpers as hlp

In [2]:
df_portfolios = pd.read_parquet(cfg.PATH_PORTFOLIO_PARQUET)

df_prelast = pd.read_parquet(cfg.PATH_PRELAST_PARQUET)
df_prelast.set_index('Data', inplace=True)

## 1. Estudando como criar os Portfolios utilizando o de _Blue Chips_ como exemplo

In [3]:
# Recuperando a lista dos ativos do portfloio
stock_list = df_portfolios['Blue Chips'].to_list()

In [4]:
# Recuperando os dados do preço dos ativos em 30/12/2019
df_initial_data = df_prelast.loc['2019-12-30']

stock_prices = df_initial_data[stock_list].to_dict()


In [5]:
# Valor inicial de cada ativo na carteira
DEFAULT_ALOC = float(10000)

# Calculando as quantidades dos ativos
stock_qtd = {stock: DEFAULT_ALOC/value for stock, value in stock_prices.items()}

In [6]:
# Criando o portfolio
df_blue_chips = pd.DataFrame(index=df_prelast.index)

for stock in stock_list:
    df_blue_chips[stock] = np.round(df_prelast[stock] * stock_qtd[stock],2)

df_blue_chips['TOTAL AMOUNT'] = df_blue_chips.sum(axis=1)

In [7]:
#Resultado
df_blue_chips.head()

,VALE3.SA,PETR4.SA,ITUB4.SA,WEGE3.SA,BBDC4.SA,BPAC11.SA,SANB4.SA,BBAS3.SA,ELET6.SA,ITSA4.SA,B3SA3.SA,SUZB3.SA,VIVT3.SA,RENT3.SA,JBSS3.SA,TOTAL AMOUNT
Data,,,,,,,,,,,,,,,,
2019-12-30,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,150000.00
2020-01-02,10193.25,10172.30,10254.82,10167.34,10370.47,10022.33,10497.24,10185.54,10122.91,10184.53,10585.18,10279.74,9989.59,10466.15,10542.64,154034.03
2020-01-03,10118.20,10089.46,10146.96,10037.51,10375.72,10028.90,10773.48,10168.50,10081.07,10035.49,10280.13,10244.46,10105.56,10314.28,10647.29,153447.01
2020-01-06,10058.16,10208.75,9995.95,10063.47,10190.39,9910.68,10171.87,10034.08,10096.76,10014.19,10181.58,10226.81,10008.23,10194.05,10395.35,151750.32
2020-01-07,10131.33,10168.99,9764.05,10141.37,10013.36,10025.96,10025.29,9958.35,10251.05,9879.35,10510.09,10357.86,10229.81,10293.19,10581.40,152331.45


## 2. Criando os portfólios e salvando em seus respectivos arquivos

In [8]:
def calculate_portifolio(portfolio_name, base_date, initial_aloc):
    # Recuperando a lista dos ativos do portfloio
    stock_list = df_portfolios[portfolio_name].to_list()

    # Recuperando os dados do preço dos ativos da data base
    stock_prices = df_prelast.loc[base_date][stock_list].to_dict()

    # Calculando o valor inicial por ativo
    initial_stock_alocation = initial_aloc / float(len(stock_list))

    # Calculando as quantidades dos ativos
    stock_qtd = {stock: np.round(initial_stock_alocation/value, 0) for stock, value in stock_prices.items()}

    # Criando o portfolio
    df_portfolio = pd.DataFrame(index=df_prelast.index)

    for stock in stock_list:
        df_portfolio[f'{stock}_qtd'] = stock_qtd[stock]
        df_portfolio[f'{stock}_price'] = df_prelast[stock]
        df_portfolio[stock] = np.round(df_prelast[stock] * stock_qtd[stock],2)

    df_portfolio['TOTAL AMOUNT'] = df_portfolio[stock_list].sum(axis=1)

    return df_portfolio

In [9]:
BASE_DATE = '2019-12-30'

# Gerando a lista de portfólios
portfolio_list = df_portfolios.columns.to_list()

# Lista para armazenar os nomes dos arquivos
file_names = []

# Valor inicial dos fundos
amount_value_fund = float(10000000)

# Para cada portfolio, calculá-lo e salvar como um parquet
for portfolio in tqdm(portfolio_list):

    df_portfolio = calculate_portifolio(portfolio, BASE_DATE, amount_value_fund)

    # Salvando o portfolio
    file_name = f"{re.sub(r'[^a-zA-Z_]', '', portfolio.replace(' ','_')).lower()}.parquet"
    hlp.save_to_parquet(df_portfolio, f"{cfg.PATH_CALCULATED_PORTFOLIO_DIR}/{file_name}")

    # Adicionando o nome do arquivo à lista
    file_names.append(file_name)

# Salvando a lista de nomes de arquivo como um arquivo CSV
with open(f'{cfg.PATH_RESULTADOS}/portfolio_files.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Portfolio', 'File Name'])
    for i, portfolio in enumerate(portfolio_list):
        writer.writerow([portfolio, file_names[i]])

  0%|          | 0/24 [00:00<?, ?it/s]